# K-Nearest Neighbors from Scratch

**Note:**  
If you want to learn more details about the algorithm, follow the blog below:

<br>

<a href="https://keoliya.hashnode.dev/simple-k-nearest-neighbors-recipe">
<img src="https://keoliya.hashnode.dev/_next/image?url=https%3A%2F%2Fcdn.hashnode.com%2Fres%2Fhashnode%2Fimage%2Fupload%2Fv1672128080283%2F02d91e86-852b-45a1-a350-92749a634f09.png%3Fw%3D1600%26h%3D840%26fit%3Dcrop%26crop%3Dentropy%26auto%3Dcompress%2Cformat%26format%3Dwebp&w=1920&q=75" height="800" width="800"/></a>

## Ingredients

In [3]:
from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
import random
from math import floor
import numpy as np

def split_dataset(dataset, test_size):
    df = dataset.sample(frac=1)
    
    X = df.drop('target', axis=1)
    y = list(df['target'])
    
    pivot = floor(len(X) * (1-test_size))
    
    X_train = np.array(X.iloc[:pivot,:])
    X_test = np.array(X.iloc[pivot:,:])
    
    y_train, y_test = y[:pivot], y[pivot:]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_dataset(df, test_size=0.25)

## Equipment

In [1]:
from math import sqrt

def euclidean_dist(vec1, vec2):
    distance = 0
    dimen = len(vec1) 
    
    for i in range(dimen):
        distance += (vec1[i] - vec2[i])**2
        
    return sqrt(distance)

In [2]:
def find_neighbors(point, X_train, y_train, k):
    distances = list()
    
    for i in range(len(X_train)):
        distance = euclidean_dist(point, X_train[i])
        distances.append((y_train[i], distance))
        
    distances.sort(key=lambda x:x[1])
    
    neighbors = list()
    
    for i in range(k):
        neighbors.append(distances[i][0])
        
    return neighbors
    

## Directions

In [5]:
def knn(X_train, X_test, y_train, y_test, k):
    y_hat = list()
    for i in range(len(X_test)):
        neighbors = find_neighbors(X_test[i], X_train, y_train, k)
        prediction = max(set(neighbors), key = neighbors.count)
        y_hat.append(prediction)
    return y_hat

In [6]:
y_hat_test = knn(X_train, X_test, y_train, y_test, 5)
print(y_hat_test)

[2, 1, 0, 2, 1, 1, 0, 2, 2, 2, 1, 1, 0, 0, 2, 2, 0, 1, 2, 0, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 0, 1, 0, 0, 1, 2, 2, 0]


## Tasting

In [7]:
def accuracy(actual, predicted):
    correct  = sum([1 for i in range(len(actual)) 
                    if actual[i]==predicted[i] ])
    return correct/len(actual)

print(accuracy(y_test,y_hat_test))

0.9473684210526315


# Advanced (Ready-Made) Method

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf = KNeighborsClassifier(n_neighbors=5, p=2)
clf.fit(X_train, y_train)
y_pred_test = clf.predict(X_test)

print(f"Sklearn KNN Accuracy: {accuracy_score(y_test, y_pred_test)}")

Sklearn KNN Accuracy: 0.9473684210526315


# Complete Code

In [22]:
from sklearn import datasets
import pandas as pd
import random
from math import floor, sqrt
import numpy as np

# ----- Preparing Training and Test Datasets ----- #
def split_dataset(dataset, test_size):
    df = dataset.sample(frac=1)
    X = df.drop('target', axis=1)
    y = list(df['target'])
    pivot = floor(len(X) * (1-test_size))
    X_train = np.array(X.iloc[:pivot,:])
    X_test = np.array(X.iloc[pivot:,:])
    y_train, y_test = y[:pivot], y[pivot:]
    return X_train, X_test, y_train, y_test

# ----- Calculating distance between points ----- #
def euclidean_dist(vec1, vec2):
    distance = 0
    dimen = len(vec1)
    for i in range(dimen):
        distance += (vec1[i]-vec2[i])**2
    return sqrt(distance)

# ----- Finding k closest points ----- 
def find_neighbors(point, X_train, y_train, k):
    distances = list()
    for i in range(len(X_train)):
        distance = euclidean_dist(point, X_train[i])
        distances.append((y_train[i], distance))      
    distances.sort(key=lambda x:x[1])
    neighbors = list()
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

# ----- Predicting class based on highest vote by neighbors ----- #
def knn_classifier(X_train, X_test, y_train, y_test, k):
    y_hat = list()
    for i in range(len(X_test)):
        neighbors = find_neighbors(X_test[i], X_train, y_train, k)
        prediction = max(set(neighbors), key = neighbors.count)
        y_hat.append(prediction)
    return y_hat

# ----- Calculating accuracy of the model ---- #
def accuracy(actual, predicted):
    correct  = sum([1 for i in range(len(actual)) 
                          if actual[i]==predicted[i]])
    return correct/len(actual)


# --------------- MAIN PROGRAM EXECUTION --------------- #

# ---- Getting data and creating splits ----- #
iris = datasets.load_iris()
dataset = pd.DataFrame(data=iris.data, columns=iris.feature_names)
dataset['target'] = iris.target
X_train, X_test, y_train, y_test = split_dataset(dataset, test_size=0.25)

# ----- Applying KNN algorithm and calculating accuracy ----- #
y_hat_test = knn_classifier(X_train, X_test, y_train, y_test, 5)
print(accuracy(y_test,y_hat_test))

0.9473684210526315
